In [46]:
import sys
sys.path.append('../../process')
import psml
import imp
import numpy as np
import imp

In [47]:
trace_dir = 'traces/'

In [48]:
imp.reload(psml)
#Load all for loop traces into one 3d array.
trc_paths = !ls traces/*act*
trcs = psml.paths_to_tracearr(trc_paths, 'act')
trcs.shape

(1, 2639, 5)

In [21]:
#Read events
t0 = trcs[0]
#% 8 for Lower 3 bits = id. id = 1 and 3 are read events
rd_events = ((t0[:,0] % 8) == 1) | ((t0[:,0] % 8) == 3)
rd_events.sum()

1839

In [22]:
#Write events
#% 8 for Lower 3 bits = id. id = 2 and 4 are write events
wr_events = ((t0[:,0] % 8) == 2) | ((t0[:,0] % 8) == 4)
wr_events.sum()

800

In [23]:
#Do they sum to entire trace?
rd_events.sum() + wr_events.sum() == trcs.shape[1]

True

In [24]:
#Trying to stay consistent with sv interpretation.
#No ir here, but is in sv at position 7. Leaving it open here.

#Reg reads
reg_rds = trcs[:,np.squeeze((trcs[:,:,0] % 8) == 3), :]

#Reg addresses
#Pc is 2 bytes. Addresses from simulator don't account for 2nd byte of pc.
reg_mask = np.unique(np.squeeze(reg_rds)[:,1]) + 1

#If Pc was one of the reads, add its 2nd byte
if 1 in reg_mask:
    reg_mask = np.append([0], reg_mask) 

#Start mem component of sv at index 8, skipping over 7.
#Mem_rds
mem_rds = trcs[:,np.squeeze((trcs[:,:,0] % 8) == 1), :]
#Mem addrs can span 2 bytes
mem_addr = np.squeeze(mem_rds)[:,1]+ 256*np.squeeze(mem_rds)[:,2]

#Shift so they start at 8
mem_mask = np.unique(np.array(mem_addr, dtype=np.uint32))+8

#reg_mask
read_mask = np.union1d(reg_mask, mem_mask)
read_mask

array([    0,     1,     2,     3,     4,     5,     6,    16,    17,
          18,    19,    24,   518,   519,   520,   521,   522,   523,
         524,   525,   526,   527,   528,   529,   530,   531,   532,
         533,   534,   535,   536,   537,   538,   539,   540,   541,
         542,   543,   544,   545,   546,   547,   548,   549,   550,
         551,   552,   553,   554,   555,   556,  1288,  1289,  1290,
        1291,  1292,  1293,  1294,  1295,  1296,  1297,  1298,  1299,
        1300,  1301,  1302,  1303,  1304,  1305,  1306,  1307,  1308,
        1309,  1310,  1311,  1312,  1313,  1314,  1315,  1316,  1317,
        1318,  1319,  1327,  1328,  1329,  1330,  1331,  1332,  1333,
        1334,  1335,  1336,  1337,  1338,  1339,  1340,  1341,  1342,
        1353,  1354,  1355,  1356,  1362,  1363,  1364,  1365,  1366,
        1367,  1368,  1369,  1370,  1378,  1379,  1380,  1381,  1382,
        1383,  1384,  1385,  1386,  1387,  1388,  1389,  1390,  1391,
        1392,  1393,

In [30]:
#Write mask
reg_writes = trcs[:,np.squeeze((trcs[:,:,0] % 8) == 4), :]
#Bump over for 2byte pc
reg_mask = np.unique(np.squeeze(reg_rds)[:,1]) + 1
#Grab 2nd byte of pc
if 1 in reg_mask:
    reg_mask = np.append([0], reg_mask) 

#Mem writes
mem_rds = trcs[:,np.squeeze((trcs[:,:,0] % 8) == 2), :]
#Mem addrs can span 2 bytes
mem_addr = np.squeeze(mem_rds)[:,1]+ 256*np.squeeze(mem_rds)[:,2]

#Shift so they start at 8
mem_mask = np.unique(np.array(mem_addr, dtype=np.uint32))+8

#Entire write mask
write_mask = np.union1d(reg_mask, mem_mask)
write_mask

array([  0,   1,   2,   3,   4,   5,   6,   8,   9,  16,  17,  18,  19,
        24, 518, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530,
       531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543,
       544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556])

In [34]:
sv_trc = np.array([psml.path_to_svarr('traces/hi.img.sv.trc')])
psml.mask_svarr(sv_trc, write_mask).shape
psml.path_to_svarr('images/hi.img')

num_states is non int!


SystemExit: 

To exit: use 'exit', 'quit', or Ctrl-D.


In [51]:
#Append zero mem state.
img = psml.img_to_sv('images/hi.img')

In [74]:
#img[8:].shape == sv_trc[0,0,8:]
(~(img == sv_trc[0,0,:])).sum()
#img[0:50]

2